In [2]:
import pandas as pd

england_water = pd.read_csv('../data/england_water_final.csv')

basin_counts = england_water['River Basin District'].value_counts()

# Identify the river basin with the most records
most_common_basin = basin_counts.idxmax()
print("Most common river basin:", most_common_basin)

# Select only the data from that river basin
england_water = england_water[england_water['River Basin District'] == most_common_basin]

print(england_water.isnull().sum())  # Kiểm tra giá trị NaN
print(england_water.describe())      # Kiểm tra giá trị bất thường

/tmp/ipykernel_2303193/1600285716.py:3: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  england_water = pd.read_csv('../data/england_water_final.csv')


Most common river basin: Humber
Unnamed: 0.2            0
Unnamed: 0.1            0
Unnamed: 0              0
datetime                0
wqp                     0
result                  0
unit                    0
station_type            0
siteEasting             0
siteNorthing            0
year                    0
month                   0
hour                    0
weekday                 0
season                  0
River Basin District    0
Management Catchment    0
Water Body ID           0
Water Body              0
Water Body Type         0
Site ID                 0
Site Region             0
dtype: int64
       Unnamed: 0.2  Unnamed: 0.1    Unnamed: 0         result    siteEasting  \
count  7.000300e+05  7.000300e+05  7.000300e+05  700030.000000  700030.000000   
mean   5.086644e+06  5.882869e+06  5.882869e+06      76.709517  441683.289558   
std    2.686528e+06  3.130144e+06  3.130144e+06     669.441570   34674.120212   
min    8.934720e+05  1.062440e+06  1.062440e+06     -17.800

In [3]:
# Just kepp the columns we need: station_id, datetime, wqp, result, unit
england_water_normalize = england_water[['Site ID', 'datetime', 'wqp', 'result']]

# Then rename the columns to match the other datasets
england_water_normalize.columns = ['station_id', 'datetime', 'metric', 'value']

# Format data type of datetime without timezone, station_id to integer
# england_water_normalize['station_id'] = england_water_normalize['station_id'].str.replace('GB-', '').astype(int)
england_water_normalize['datetime'] = pd.to_datetime(england_water_normalize['datetime']).dt.tz_localize(None)

england_water_normalize.head(5)

/tmp/ipykernel_2303193/4196422668.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  england_water_normalize['datetime'] = pd.to_datetime(england_water_normalize['datetime']).dt.tz_localize(None)


,station_id,datetime,metric,value
192930,1161,2018-10-03 09:35:00,Alky pH 4.5,225.0
192931,1161,2018-07-05 12:32:00,Alky pH 4.5,268.0
192932,1161,2018-05-02 14:35:00,Alky pH 4.5,281.0
192933,1161,2018-06-12 09:56:00,Alky pH 4.5,272.0
192934,1161,2018-11-06 09:55:00,Alky pH 4.5,239.0


In [6]:
england_water_normalize.info()

<class 'pandas.core.frame.DataFrame'>
Index: 700030 entries, 192930 to 3252897
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   station_id  700030 non-null  object        
 1   datetime    700030 non-null  datetime64[ns]
 2   metric      700030 non-null  object        
 3   value       700030 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 26.7+ MB


In [7]:
# Just keep the rows where station_id is integer not a string
england_water_normalize = england_water_normalize[england_water_normalize['station_id'].astype(str).apply(lambda x: x.isnumeric())]
# Convert station_id to integer
england_water_normalize['station_id'] = england_water_normalize['station_id'].astype(int)
england_water_normalize.info()

<class 'pandas.core.frame.DataFrame'>
Index: 672874 entries, 192930 to 1709664
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   station_id  672874 non-null  int64         
 1   datetime    672874 non-null  datetime64[ns]
 2   metric      672874 non-null  object        
 3   value       672874 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 25.7+ MB


In [ ]:
import pyarrow.csv as pa_csv
import pyarrow.parquet as pq
import subprocess

## england_water_normalize is pandas DataFrame and I want to convert it to parquet file by england_water_normalize 
england_water_normalize.to_parquet('../data/england_water.parquet', index=False)

cmd = f'cat ../data/england_water.parquet | clickhouse-client --host 160.191.49.128 --port 9100 --user default --password shard1 --query="INSERT INTO station_metrics.messages_sharded format Parquet"'
subprocess.run(cmd, shell=True, check=True)

AttributeError: 'DataFrame' object has no attribute 'schema'